<a href="https://colab.research.google.com/github/hoavu-cs/notebooks/blob/main/tiny_english_stories_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import random
import pickle
import math
from tqdm import tqdm
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
from transformers import AutoTokenizer, BertTokenizer
from sklearn.model_selection import train_test_split
import gc
from tokenizers import ByteLevelBPETokenizer, processors

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [1]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.7 MB/s eta 0:00:00


In [4]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')
!ls "/content/drive/My Drive/datasets/NLP/tiny_stories"
data_path = "/content/drive/My Drive/datasets/NLP/tiny_stories"


Mounted at /content/drive
english_tiny_stories_18m.pth  merges.txt
english_tiny_stories_28m.pth  TinyStoriesV2-GPT4-train.txt
english_tiny_stories_37m.pth  TinyStoriesV2-GPT4-valid.txt
english_tiny_stories.pth      vocab.json


In [99]:
filename = 'TinyStoriesV2-GPT4-train.txt'
val_filename = 'TinyStoriesV2-GPT4-valid.txt'

filepath = os.path.join(data_path, filename)
with open(filepath, 'r', encoding='utf-8') as file:
    train_data = file.read()
train_data = train_data.replace('\n', '')
train_data = train_data.split('<|endoftext|>')

val_filepath = os.path.join(data_path, val_filename)
with open(val_filepath, 'r', encoding='utf-8') as file:
    val_data = file.read()
val_data = val_data.replace('\n', '')
val_data = val_data.split('<|endoftext|>')

print(train_data[0])





Once upon a time there was a little boy named Ben. Ben loved to explore the world around him. He saw many amazing things, like beautiful vases that were on display in a store. One day, Ben was walking through the store when he came across a very special vase. When Ben saw it he was amazed!  He said, “Wow, that is a really amazing vase! Can I buy it?” The shopkeeper smiled and said, “Of course you can. You can take it home and show all your friends how amazing it is!”So Ben took the vase home and he was so proud of it! He called his friends over and showed them the amazing vase. All his friends thought the vase was beautiful and couldn't believe how lucky Ben was. And that's how Ben found an amazing vase in the store!


In [125]:
train_data = pd.DataFrame(train_data, columns=['content'])
val_data = pd.DataFrame(val_data, columns=['content'])

print(train_data.shape)
print(val_data.shape)

print(train_data.head())


(2717700, 1)
(27631, 1)
                                             content
0  Once upon a time there was a little boy named ...
1  Once upon a time, there was a reliable otter n...
2  One day, a little boy named Tim went to the pa...
3  Once upon a time there was a friendly little b...
4  Once upon a time, in a small house, there live...


In [101]:
tokenizer = ByteLevelBPETokenizer()

tokenizer.train(files=[os.path.join(data_path, 'TinyStoriesV2-GPT4-valid.txt')], vocab_size=10000, min_frequency=1, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

tokenizer.save_model(data_path)

['/content/drive/My Drive/datasets/NLP/tiny_stories/vocab.json',
 '/content/drive/My Drive/datasets/NLP/tiny_stories/merges.txt']

In [105]:
vocab_path = os.path.join(data_path, "vocab.json")
merge_path = os.path.join(data_path, "merges.txt")

tokenizer = ByteLevelBPETokenizer(
    vocab=vocab_path,
    merges=merge_path
)

tokenizer.add_special_tokens(["<s>", "<pad>", "</s>", "<unk>", "<mask>"])

tokenizer._tokenizer.post_processor = processors.BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)



In [132]:
# 4. Encode a text string
print(train_data.loc[0, 'content'])
output = tokenizer.encode(train_data.loc[0, 'content']).ids
print("Decoded string: ", tokenizer.decode(output, skip_special_tokens=False))  # decoding back to the original string

Once upon a time there was a little boy named Ben. Ben loved to explore the world around him. He saw many amazing things, like beautiful vases that were on display in a store. One day, Ben was walking through the store when he came across a very special vase. When Ben saw it he was amazed!  He said, “Wow, that is a really amazing vase! Can I buy it?” The shopkeeper smiled and said, “Of course you can. You can take it home and show all your friends how amazing it is!”So Ben took the vase home and he was so proud of it! He called his friends over and showed them the amazing vase. All his friends thought the vase was beautiful and couldn't believe how lucky Ben was. And that's how Ben found an amazing vase in the store!
Decoded string:  <s>Once upon a time there was a little boy named Ben. Ben loved to explore the world around him. He saw many amazing things, like beautiful vases that were on display in a store. One day, Ben was walking through the store when he came across a very special

In [10]:
BLOCK_SIZE = 256
VOCAB_SIZE = len(tokenizer.get_vocab())
print('Vocab size is ', VOCAB_SIZE)


Vocab size is  10000


In [83]:
eot_id = tokenizer.token_to_id("<|endoftext|>")
start_id = tokenizer.token_to_id("<s>")
end_id = tokenizer.token_to_id("</s>")
pad_id = tokenizer.token_to_id("<pad>")
unk_id = tokenizer.token_to_id("<unk>")

In [175]:
def get_batch(data, block_size, batch_size):
    x = torch.zeros((batch_size, block_size), dtype=torch.long)
    y = torch.zeros((batch_size, block_size), dtype=torch.long)
    samples = data['content'].sample(n=batch_size)

    for i, sample in enumerate(samples):
        summary_ids = tokenizer.encode(sample).ids
        #summary_ids = [id for id in summary_ids if id != UNK_ID]
        if len(summary_ids) < block_size + 2:
            summary_ids = summary_ids + [pad_id] * (block_size + 2 - len(summary_ids))
        random_start = random.randint(0, len(summary_ids) - block_size - 2)
        x[i, :len(summary_ids)] = torch.tensor(summary_ids[random_start:random_start + block_size], dtype=torch.long)
        y[i, :len(summary_ids)] = torch.tensor(summary_ids[random_start + 1:random_start + block_size + 1], dtype=torch.long)

    return x, y

a, b = get_batch(train_data, block_size=BLOCK_SIZE, batch_size=1)
print(a.shape, b.shape)
print(tokenizer.decode(a[0].tolist(), skip_special_tokens=False))
print(tokenizer.decode(b[0].tolist(), skip_special_tokens=False))

torch.Size([1, 256]) torch.Size([1, 256])
<s>Once upon a time, there was a little girl named Lucy. She loved to open boxes and find surprises inside. One day, Lucy found a big, dark box in her room. She was very excited and wanted to see what was inside.Lucy opened the dark box and saw many colorful treats. She wanted to taste them all. She picked up a red treat and took a small bite. It was sweet and yummy! Then, she tried a green treat. It was a little sour, but she liked it too.Lucy shared the treats with her friends. They all enjoyed the different tastes. In the end, the dark box was empty, but Lucy and her friends were happy. They had fun trying new treats and playing together. And that's the end of the story.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

In [ ]:
del model  # delete the tensor variable
torch.cuda.empty_cache()  # clear unused memory in PyTorch
gc.collect()  # call Python garbage collector

583

In [154]:
N_EMB = 1200
N_LAYERS = 4
N_HEADS = 4
DROPOUT = 0.2


def estimate_loss(model, val_data, block_size, batch_size):
    model.eval()
    with torch.no_grad():
        x, y = get_batch(val_data, block_size, batch_size)
        x, y = x.to(device), y.to(device)
        _, loss = model(x, y)
    model.train()
    return loss.item()

def generate_square_subsequent_mask(sz):
    mask = (torch.tril(torch.ones(sz, sz)) == 1).float()
    mask = mask.masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def get_sine_position_encodings(length, dim):
    pos = torch.arange(length, dtype=torch.float32).reshape(-1, 1)
    div_term = torch.exp(torch.arange(0, dim, 2).float() * -(math.log(10000.0) / dim))
    pos_encodings = torch.zeros(length, dim)
    pos_encodings[:, 0::2] = torch.sin(pos * div_term)
    pos_encodings[:, 1::2] = torch.cos(pos * div_term)
    return pos_encodings

class LanguageModel(nn.Module):

    def __init__(self, vocab_size, n_emb, block_size, n_layers, n_heads, dropout=0.2):
        super(LanguageModel, self).__init__()

        self.token_embedding_table = nn.Embedding(vocab_size, n_emb)
        self.position_embedding_table = nn.Embedding(block_size, n_emb)

        encoder_layer = nn.TransformerEncoderLayer(d_model=n_emb, nhead=n_heads, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)

        self.feed_forward = nn.Sequential(
            nn.Linear(n_emb, 6 * n_emb),
            nn.ReLU(),
            nn.Linear(6 * n_emb, n_emb)
        )

        self.lm_head = nn.Linear(n_emb, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        token_emb = self.token_embedding_table(idx)
        position_emb = self.position_embedding_table(torch.arange(T, device=device))

        x = token_emb + position_emb
        x_transform = x.clone()
        mask = generate_square_subsequent_mask(T).to(device)

        x_transform = self.transformer_encoder(x_transform.permute(1, 0, 2), mask=mask)
        x_transform = x_transform.permute(1, 0, 2)
        x = x + x_transform

        x = self.feed_forward(x)
        logits = self.lm_head(x)

        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
            return logits, loss
        else:
            return logits, None


    def generate(self, idx, max_new_tokens, block_size, temperature=1.0, stop_token=False):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self.forward(idx_cond)

            # Scale logits by the temperature
            logits = logits[:, -1, :] / temperature

            probs = F.softmax(logits, dim=-1)
            idx_new = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, idx_new], dim=-1)
            if stop_token and idx_new.item() == end_id:
                break
        return idx

# Create model, optimizer
model = LanguageModel(vocab_size=VOCAB_SIZE, block_size=BLOCK_SIZE, n_emb=N_EMB, n_layers=N_LAYERS, \
    n_heads=N_HEADS, dropout=DROPOUT).to(device)

print(f'Number of parameters {sum(p.numel() for p in model.parameters() if p.requires_grad)}')


Number of parameters 84357792


In [ ]:
model_path = os.path.join(data_path, "english_tiny_stories.pth")
model = torch.load(model_path)

In [190]:
EARLY_STOP = 50
N_EPOCHS = 1000
BATCH_SIZE = 32
LEARNING_RATE = 3e-4
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

last_val_loss = 1e9
early_stop = EARLY_STOP

for steps in range(N_EPOCHS):
    model.train()
    xb, yb = get_batch(train_data, block_size=BLOCK_SIZE, batch_size=BATCH_SIZE)
    xb = xb.to(device)
    yb = yb.to(device)
    logits, loss = model(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Delete xb, yb and free GPU memory
    del xb, yb
    torch.cuda.empty_cache()

    if steps % 100 == 0:
        print('Step:', steps, 'Training Loss:', loss.item())
        val_loss = estimate_loss(model, val_data, block_size=BLOCK_SIZE, batch_size=BATCH_SIZE)
        print('Validation loss:', val_loss)
        if val_loss >= last_val_loss:
            early_stop -= 1
            if early_stop == 0:
                print('Early stop!')
                break
        else:
            early_stop = EARLY_STOP
            last_val_loss = val_loss

Step: 0 Training Loss: 1.4466464519500732
Validation loss: 1.4531854391098022
Step: 100 Training Loss: 1.4212156534194946
Validation loss: 1.5039832592010498
Step: 200 Training Loss: 1.4404656887054443
Validation loss: 1.3775882720947266
Step: 300 Training Loss: 1.3986014127731323
Validation loss: 1.6962032318115234
Step: 400 Training Loss: 1.4141086339950562
Validation loss: 1.393838882446289
Step: 500 Training Loss: 1.5298128128051758
Validation loss: 1.2274142503738403
Step: 600 Training Loss: 1.5050522089004517
Validation loss: 1.6843595504760742
Step: 700 Training Loss: 1.599587321281433
Validation loss: 1.4229384660720825
Step: 800 Training Loss: 1.4752652645111084
Validation loss: 1.4078549146652222
Step: 900 Training Loss: 1.3300975561141968
Validation loss: 1.2947711944580078


In [187]:
model_path = os.path.join(data_path, "english_tiny_stories_2.pth")
torch.save(model, model_path)

In [193]:
starting_tokens = 'Once upon a time, Alice lost her dog.'

encoded_start = tokenizer.encode(starting_tokens).ids
encoded_start.pop(-1)
len_starting_tokens = len(encoded_start)

idx = torch.tensor(encoded_start).reshape(1, len_starting_tokens).to(device)
model.eval()
N_SAMPLES = 5
for _ in range(N_SAMPLES):
    generation = model.generate(idx, max_new_tokens=500, block_size=BLOCK_SIZE, temperature=0.5, stop_token=True)[0].tolist()
    story = tokenizer.decode(generation, skip_special_tokens=True)

    print('Story ', _ + 1, ':')
    print(story)
    print('\n')






Story  1 :
Once upon a time, Alice lost her dog. She was very sad and missed the dog. She looked everywhere but could not find her dog. She looked everywhere but could not find it. She was very sad.One day, Alice's mommy saw her looking sad. She knew she did not have her dog. She asked her mommy, where her mommy said, "Why is your dog?" Her mommy said, "I am sad because I have no friends to play."Alice helped her mommy find her friends. They looked under the bed, behind the couch, and in the yard. They looked under the bed, behind the couch, and in the kitchen. They found her dog and her dog. They were very happy.Alice learned that she could be happy too.


Story  2 :
Once upon a time, Alice lost her dog. She was very sad and missed her dog. She wanted to find her friend. She looked everywhere in the park. She looked under the trees and behind the trees. She looked near the swings and behind the swings, and in the sandbox. Finally, she found her friend, the squirrel. "Look, squirrel! I